<font size= 6>**Market Sizing and Forecasting Related**</font></p>

# About this program
- Purpose: for checking market sizes with different US or regional analysts during the market forecast process or for post release analysis
- Output: write out to multi-sheet excel file(s) that contains services market forecast data from current version (live data), last cycle's version, central modeling forecast model, cloud segmentation, etc.; each foundation market as a sheet
- More outputs: write to excel more custom market views 

## Code workflow:
- import libs
- functions (if needed)
- read from Ferda
- iteration: create multiple sheets, create pivots, write to sheets

# Import Libraries

In [1]:
import os

import numpy as np
import pandas as pd
import xlwings as xw
from IPython.core.display import display

# catpy libs (clases developed by the custom research group)
from catpy.io import get_ferda     ### built on ferpy
from catpy import Data as dd

%config IPCompleter.greedy=True   # this is for tab completion in jupyter lab/notes

# Read Data from Ferda
- import ferpy
- init and pass params
- create object
- save to csv and load into df
- close ferda link

### Where to find module ID

* *In Ferda Admin*: http://frd-prod-webapps.aws.insideidc.com:3000/home Log in with your username and password you use in Ferda. On the left panel, select *Self service*. On the page you can enter the name of the module into *Module name* and click *Submit*. It will show you the list of the modules with same or similar name. Find the one you are looking for and click *See details*. New page will open for you with information about the module. In the basic information you can see the ID of the module.

## Input Parameters Needed for Reading Data from Ferda

Mainly, we need module and snapshot <br>

Market forecast module info:
- module name: Services semiannual Fcst Segmentation (2021Q2) 
- module id: services_semiannual__8831
<br><br>
- module name: Services Fcst Semiannual Segmentation CFM-Complete
- module id: services_semiannual__209826

<br>
last cycle's snapshot: snap541572

In [2]:
# set module, snapshot info

forecast_module_id = "services_semiannual__8831"
last_c_snapshot_id = "snap541572"

forecast_cfm_module_id = 'services_semiannual__209826'

## Initiate and Read from Ferda

In [3]:
# import ferpy library
from ferpy.ferpy import FerPy

# Initiate FerPy and pass parameters as dictionary (Services Semiannual Tracker)
## this is for logging into ferda (authentication...)
## temporary folder is where the data file is kept; mainly use the data for loading dataframes - no real use for actual data files
init_params = {"Environment": "PRODUCTION",
               "Username": "xzhang",
               "Temporary folder": "C:/IDC_XF_Python/FerPy_Related/",
               "Log to console": True,
               "Cache time": 60,
               "Cache file": "./cache_file.json"
              }

####### the following codes are the standard FerPy codesthese codes are FerPy codes
    ###### normall you need to use ferda = FerPay(.....) to create an object and use FerPy's methods
        ##### however, since I am using catpy's get_ferda method, I don't need to run it here
        ##### catpy method built on this make it even easier but you need learn and use their classes
        
# ferda = FerPy(user_data_file = init_params, technology = forecast_module_id)
# ferda_cfm = FerPy(user_data_file = init_params, technology = forecast_cfm_module_id)

### use captpy's get_ferda method to read data from ferda
- pull data from live/current version, regional source, then
- last cycles
- finally from the CFM module and pull out data

In [5]:
%%time

# Market forecast module:
## module name: Services semiannual Fcst Segmentation (2021Q2)
## module id: services_semiannual__8831
# this may take up to a minute or two, depending on your connection, equipment

# specify attributes to download (mkt, geo, etc.)
ferda_attrs = ["Region", "Country", 
               "Primary Market", "Secondary Market", "Foundation Market", "Sub-Foundation Market", 
               "Delivery Type",
               "Year"]

# Download Ferda data as a dataframe
result = get_ferda(
    ferda_id=forecast_module_id,
    selected_attrs=ferda_attrs,
    user_filter={
        "Feedback": "Regional Source",
        "Year": [str(x) for x in range(2018, 2026)]
    },
    figures=["Value", "Value (Constant)"]
)

### Last cycle's data from regional serivces
# Download Ferda data as a dataframe
result_lc = get_ferda(
    ferda_id=forecast_module_id,
    selected_attrs=ferda_attrs,
    snapshot=last_c_snapshot_id,
    user_filter={
        "Feedback": "Regional Source",
        "Year": [str(x) for x in range(2018, 2026)]
    },
    figures=["Value", "Value (Constant)"]
)

## ww feedback data - this has PEOST numbers
# Download Ferda data as a dataframe
result_ww_source = get_ferda(
    ferda_id=forecast_module_id,
    selected_attrs=ferda_attrs,
    user_filter={
        "Feedback": "WW Source",
        "Year": [str(x) for x in range(2018, 2026)]
    },
    figures=["Value", "Value (Constant)"]
)

## central forecast model target (projected market size & growth)
# Download Ferda data as a dataframe
result_cfm = get_ferda(
    ferda_id=forecast_cfm_module_id,
    selected_attrs=ferda_attrs,
    user_filter={
        "Feedback": "Forecast Model Source",
        "Year": [str(x) for x in range(2018, 2026)]
    },
    figures=["Value"]   ### CFM module only has value - it's predicting the future, where value == value in cc (constant currency)
)

Wall time: 52.1 s


In [7]:
# usually, if you are using the standard ferpy class and methods, you need to close ferda link: ferda.done
## data is alreayd loaded into dfs; therefore, you want to close the link to ferda
## this is a good practice and also avoids log files causing problems in the future, but since we are using catpy's, I suppose that there is no need to do so...

# ferda.done()

# Simplify and Shorten: Some Quick Transformation
- use acronyms to replace geo and market

In [8]:
result

,Region,Country,Primary Market,Secondary Market,Foundation Market,Sub-Foundation Market,Delivery Type,Year,Value,Value (Constant)
0,Central & Eastern Europe,Rest of CEE,Support Services,IT Education and Training,IT Education and Training,IT Education and Training,Cloud-Related,2022,39.74,39.74
1,APeJC,Thailand,Managed Services,Business Process Outsourcing,Key Horizontal BPO,Finance and Accounting Business Process Outsou...,Cloud-Related,2022,22.23,22.23
2,Central & Eastern Europe,Czech Republic,Support Services,IT Education and Training,IT Education and Training,IT Education and Training,Cloud-Related,2022,19.83,19.83
3,Western Europe,Ireland,Support Services,IT Deploy and Support,Hardware Deploy and Support,Hardware Deploy and Support (ex. Printers and ...,non-Cloud,2022,75.68,75.68
4,APeJC,Taiwan,Project Oriented,Systems and Network Implementations,Network Consulting and Integration,Network Consulting and Integration,Cloud-Related,2022,64.03,64.03
...,...,...,...,...,...,...,...,...,...,...
15856,Western Europe,Germany,Managed Services,Business Process Outsourcing,Key Horizontal BPO,Customer Care Business Process Outsourcing,Cloud-Related,2021,131.62,131.61
15857,APeJC,Taiwan,Managed Services,Business Process Outsourcing,Key Horizontal BPO,Finance and Accounting Business Process Outsou...,Cloud-Related,2021,16.15,16.17
15858,Latin America,Argentina,Support Services,IT Deploy and Support,Hardware Deploy and Support,Hardware Deploy and Support (ex. Printers and ...,non-Cloud,2021,95.44,94.09
15859,Central & Eastern Europe,Czech Republic,Support Services,IT Deploy and Support,Software Deploy and Support,Software Deploy and Support,non-Cloud,2021,262.95,263.88


In [6]:
# Read results int dataframes with more appropriate param names
## b/c i will be using catpy's pivot method, it would be easier to have just one "value" column
### splitting dataframes into value dfs and value constant dfs
##### if a dataframe has cc (constant currency) at the end, then it is value (constant); otherwise, assume that it is value

df = result.drop("Value (Constant)", axis = 1)
df_cc = result.drop("Value", axis = 1)

df_last_cycle = result_lc.drop("Value (Constant)", axis = 1)
df_last_cycle_cc = result_lc.drop("Value", axis = 1)

df_ww_source = result_ww_source.drop("Value (Constant)", axis = 1)
df_ww_source_cc = result_ww_source.drop("Value", axis = 1)

df_cfm = result_cfm

In [7]:
# signing regions abbreviation
# market name abbreviation

def abbr_region_market(df, region, market):
    
    region_abbr = {
        'Central & Eastern Europe' : 'CEE',
        'Japan' : 'JP',
        'Latin America' : 'LA',
        'Middle East & Africa' : 'MEA',
        'Western Europe' : 'WE',
        'APeJC' : 'APeJC',
        'USA' : 'USA',
        'Canada' : 'CA', 
        'PRC' : 'PRC'}

    f_mkt_abbr = {
        "Application Management" : "AM", 
        "Business Consulting" : "BC", 
        "Custom Application Development" : "CAD", 
        "Hardware Deploy and Support" : "HD&S",
        "Key Horizontal BPO" : "Horizontal BPO", 
        "Hosted Application Management" : "HAM", 
        "Hosting Infrastructure Services" : "HIS",
        "IT Consulting" : "ITC", 
        "IT Education and Training" : "IT E&T", 
        "IT Outsourcing" : "ITO", 
        "Network and Endpoint Outsourcing Services" : "NEOS",
        "Network Consulting and Integration" : "NC&I", 
        "Product Engineering Services" : "PEOTS", 
        "Software Deploy and Support" : "SD&S", 
        "Systems Integration" : "SI"}
    
    df.replace({region : region_abbr,
                market: f_mkt_abbr}, inplace = True)
    return df

df = abbr_region_market(df, 'Region', 'Foundation Market')
df_cc = abbr_region_market(df_cc, 'Region', 'Foundation Market')

df_last_cycle = abbr_region_market(df, 'Region', 'Foundation Market')
df_last_cycle_cc = abbr_region_market(df_last_cycle_cc, 'Region', 'Foundation Market')

df_ww_source = abbr_region_market(df_ww_source, 'Region', 'Foundation Market')
df_ww_source_cc = abbr_region_market(df_ww_source_cc, 'Region', 'Foundation Market')

df_cfm = abbr_region_market(df_cfm, 'Region', 'Foundation Market')

In [8]:
# remove PEOTS numbers

def exclude_market(df, mkt_to_exclude):
    df = df[-df['Foundation Market'].isin(mkt_to_exclude)]
    return df

mkt_to_excl = ['PEOTS']

df = exclude_market(df, mkt_to_excl)
df_cc = exclude_market(df_cc, mkt_to_excl)
df_last_cycle = exclude_market(df_last_cycle, mkt_to_excl)
df_last_cycle_cc = exclude_market(df_last_cycle_cc, mkt_to_excl)
df_ww_source = exclude_market(df_ww_source, mkt_to_excl)
df_ww_source_cc = exclude_market(df_ww_source_cc, mkt_to_excl)
df_cfm = exclude_market(df_cfm, mkt_to_excl)

# Write to Excel

## Stationaries & Tools

### Create multiple labels (write to excel)

In [9]:
abt_current_cycle = "Current Cycle - Live Data: Published/To Be Published This Cycle"
abt_last_cycle = "Last Cycle: Data Actually Published During Previous Release Cycle"
abt_current_vs_last = "Current vs Last Cycle (% Differences)"
abt_CFM = "CFM (Central Forecast Model): Market Size Projected by the Statistical Modeling Tool (Time Series + Regression)"
abt_cloud_segment = "Cloud Segmentation: Cloud Related vs Non-Cloud"

tbl_name_value = "Revenue (US$M)"
tbl_name_grwth = "YoY Growth Rate (%)"
tbl_name_share = "Share (%)"
tbl_name_diff = "Current vs Last Cycle Release - % Difference"
tbl_name_grwth_diff = "Current vs Last Cycle Release - YoY Growth Difference"

### Create write to excel functions

In [10]:
# Define re-used functionality for writing to excel and formatting accordingly

def set_page_title(sheet, title, loc):
    sheet.range(loc).options(index=False).value = title
    tit_format = sheet.range(loc)
    tit_format.api.Font.Bold = True
    tit_format.api.Font.Size = 19
    # tit_format.api.Font.Color = 0xFF   # this is red
    tit_format.api.Font.Color = 0x0000ff   # this should be blue

def set_page_sub_title(sheet, title, loc):
    sheet.range(loc).options(index=False).value = title
    tit_format = sheet.range(loc)
    tit_format.api.Font.Bold = True
    tit_format.api.Font.Size = 14
    tit_format.api.Font.Color = 0x4b0082
    # tit_format.api.Font.italic = True

def write_table_name(sheet, loc, table_name):
    sheet.range(loc).value = table_name
    table_name_range = sheet.range(loc)
    table_name_range.api.Font.Bold = True
    table_name_range.api.Font.Size = 12
    table_name_range.api.NumberFormat = "General"

def write_table(sheet, loc, df_, num_fmt):
    sheet.range(loc).value = df_
    tbl_range = sheet.range(loc).expand("table")
    tbl_range.api.NumberFormat = num_fmt
    tb_head = sheet.range(loc).expand("right")
    tb_head.color = (0, 51, 102)
    tb_head.api.Font.Bold = True
    tb_head.api.Font.Color = 0xFFFFFF
    # This set the column names from 2,021 to just 2021
    # (change format from number to general)
    tb_head.api.NumberFormat = "General"

## Loop and Write to excel
- functions
- loop through foundation markets
- create sheet and name the sheet
- build pivots
- write to excel
- loop back to the next item - market

<font size= 3>**YOU NEED TO HAVE XLWINGS API RUNNING**</font></p>

In [14]:
# create xlwings object to open excel
file_path = f"C:/Users/{os.getlogin()}/OneDrive - IDC/Documents/Tracker and Others/Market Forecast/2021H1 Market Forecast/2021H1 cycle market forecast vs last cycle.xlsx"
wb = xw.Book()

### Write to Excel

#### Each Market by Region, df_cfmConstant Currency

In [15]:
#### All in constant currency!

# create list of foundation markets - sheet names
markets = sorted(df_cc["Foundation Market"].unique())
# markets

# Loop through and create 4 tables for each sheet and write to each sheet
## I am using catpy's pivot method to streamline the codes, then groupby
## and simply add a sum row at the bottom

for market in list(reversed(markets)):
    
    # create new sheet, load sheet object
    wb.sheets.add(market)
    sht = wb.sheets(market)
    
    # try:
    ##### all in constant currency #######

    # this cycle - value
    df_pivot_cc = dd.pivot(df_cc[df_cc['Foundation Market']==market])
    df_pivot_cc = df_pivot_cc.groupby(['Region']).sum()
    df_pivot_cc.loc['Total'] = df_pivot_cc.sum()

    # this cycle growth
    df_gwth_pivot_cc = df_pivot_cc.pct_change(axis=1)

    # last cycle - value
    df_last_cycle_pivot_cc = dd.pivot(df_last_cycle_cc[df_last_cycle_cc['Foundation Market']==market])
    df_last_cycle_pivot_cc = df_last_cycle_pivot_cc.groupby(['Region']).sum()
    df_last_cycle_pivot_cc.loc['Total'] = df_last_cycle_pivot_cc.sum()

    # last cycle growth
    df_last_cycle_gwth_pivot_cc = df_last_cycle_pivot_cc.pct_change(axis=1)

    # current vs previous cycle - % diff of value
    df_diff_pct_pivot_cc = (df_pivot_cc - df_last_cycle_pivot_cc)/df_last_cycle_pivot_cc

    # current vs last cycle: growth percent difference
    df_gwth_diff_pivot_cc = df_gwth_pivot_cc - df_last_cycle_gwth_pivot_cc

    # projected/modeled (cfm) value
    df_cfm_pivot = dd.pivot(df_cfm[df_cfm['Foundation Market']==market])
    df_cfm_pivot = df_cfm_pivot.groupby(['Region']).sum()
    df_cfm_pivot.loc['Total'] = df_cfm_pivot.sum()

    # projected/modeled (cfm) grwth (%)
    df_cfm_gwth_pivot = df_cfm_pivot.pct_change(axis=1)

    # cloud segment - current cycle
    df_mkt_cc = df_cc[df_cc['Foundation Market']==market]  
    # not sure if this is the best way but cross tab does won't let me filter df in function 
    # cloud/non-cloud split - value
    df_cloud_cross_tab_cc = pd.crosstab(
        [df_mkt_cc.Region, df_mkt_cc['Delivery Type']], 
        df_mkt_cc.Year, 
        values = df_mkt_cc["Value (Constant)"], 
        aggfunc = sum, 
        margins = False,
        rownames = ['Region', "Cloud/Non-Cloud"], 
        colnames = ['Year'],)
        # normalize = False)    #### don't think we really need this for the sub-total % calculation

    # cloud/non-cloud grwth (%)
    df_cloud_gwth_cross_tab_cc = df_cloud_cross_tab_cc.pct_change(axis=1)

    # cloud/non-cloud share of total (%)
    # calculate the total first
    df_cross_tab_cc = pd.crosstab(
        [df_mkt_cc.Region], 
        df_mkt_cc.Year, 
        values = df_mkt_cc["Value (Constant)"], 
        aggfunc = sum, 
        margins = False,
        rownames = ['Region'], 
        colnames = ['Year'],)
        # normalize = False)    #### don't think we really need this for the sub-total % calculation     
    # then divide cloud/non-cloud by total
    df_cloud_pct_cross_tab_cc = df_cloud_cross_tab_cc / df_cross_tab_cc


    # Writing to excel
    ### - read sheet object

    ### - write label: current cycle
    ### - write table 1 (left), format the table, format header
    ### - write table 2 (right), format the table, format header

    ### - write label: last cycle
    ### - write table 3 (left), format the table, format header
    ### - write table 4 (right), format the table, format header

    ### - write label: current vs last cycle
    ### - write table 5 (left), format the table, format header
    ### - write table 6 (right), format the table, format header

    ### - write label CFM
    ### - write table 7 (left), format the table, format header
    ### - write table 8 (right), format the table, format header


    ### - write label: cloud segmentation

    ### - write table 7 (left), cloud segment - share, format table, format header



    # NOTE: The separate + 1 in the calculation of the range is to account for the header of the table

    # set first tab location
    # position the tables\
    lef_tbl_cell = (5, 1)  # set the position of table 1 (left)

    # title - foundation market
    set_page_title(sht, market, 'A1')
    set_page_sub_title(sht, "All revenue figures are in CC (Constant Currency) except for CFM Data", 'G1')

    # table 1 & 2 section label: current cycle
    set_page_sub_title(sht, abt_current_cycle, "A3")

    # write table 1
    write_table_name(sht, ((lef_tbl_cell[0]-1), lef_tbl_cell[1]), tbl_name_value)
    write_table(sht, lef_tbl_cell, df_pivot_cc, "#,0")

    # write table 2
    rght_tbl_cell = (lef_tbl_cell[0], df_pivot_cc.shape[1] + 1 + 3)  ### shape[0] is row and shape[1] is column
    write_table_name(sht, ((rght_tbl_cell[0]-1), rght_tbl_cell[1]), tbl_name_grwth)
    write_table(sht, rght_tbl_cell, df_gwth_pivot_cc, ".0%")

    # table 3 & 4 section label: last cycle
    set_page_sub_title(sht, abt_last_cycle, (lef_tbl_cell[0] + df_pivot_cc.shape[0] + 2, 1))

    # write table 3
    lef_tbl_cell = (lef_tbl_cell[0] + df_pivot_cc.shape[0] + 1 + 3, 1)
    write_table_name(sht, ((lef_tbl_cell[0]-1), lef_tbl_cell[1]), tbl_name_value)    
    write_table(sht, lef_tbl_cell, df_last_cycle_pivot_cc, "#,0")

    # write table 4
    rght_tbl_cell = (lef_tbl_cell[0], df_last_cycle_pivot_cc.shape[1] + 1 + 3)  ### shape[0] is row and shape[1] is column
    write_table_name(sht, ((rght_tbl_cell[0]-1), rght_tbl_cell[1]), tbl_name_grwth)
    write_table(sht, rght_tbl_cell, df_last_cycle_gwth_pivot_cc, ".0%")

    # table 5 & 6 section label: current vs last cycle
    set_page_sub_title(sht, abt_current_vs_last, (lef_tbl_cell[0] + df_last_cycle_pivot_cc.shape[0] + 2, 1))

    # write table 5: current vs last % difference    
    lef_tbl_cell = (lef_tbl_cell[0] + df_last_cycle_pivot_cc.shape[0] + 1 + 3, 1)
    write_table_name(sht, ((lef_tbl_cell[0]-1), lef_tbl_cell[1]), tbl_name_diff)
    write_table(sht, lef_tbl_cell, df_diff_pct_pivot_cc, ".0%")

    # write table 6
    rght_tbl_cell = (lef_tbl_cell[0], df_diff_pct_pivot_cc.shape[1] + 1 + 3)        ### shape[0] is row and shape[1] is column
    write_table_name(sht, ((rght_tbl_cell[0]-1), rght_tbl_cell[1]), tbl_name_grwth)
    write_table(sht, rght_tbl_cell, df_gwth_diff_pivot_cc, ".0%")    

    # table 7 & 8 section label: CFM
    set_page_sub_title(sht, abt_CFM, (lef_tbl_cell[0] + df_diff_pct_pivot_cc.shape[0] + 2, 1))

    # write tabe 7: CFM value
    lef_tbl_cell = (lef_tbl_cell[0] + df_diff_pct_pivot_cc.shape[0] + 1 + 3, 1)     ### shape[0] is row and shape[1] is column
    write_table_name(sht, ((lef_tbl_cell[0]-1), lef_tbl_cell[1]), tbl_name_value)    
    write_table(sht, lef_tbl_cell, df_cfm_pivot, "#,0")

    # write table 8
    rght_tbl_cell = (lef_tbl_cell[0], df_cfm_pivot.shape[1] + 1 + 3)     ### shape[0] is row and shape[1] is column
    write_table_name(sht, (rght_tbl_cell[0]-1, rght_tbl_cell[1]), tbl_name_grwth)
    write_table(sht, rght_tbl_cell, df_cfm_gwth_pivot, ".0%")

    # table 9, 10, 11 section label: cloud related
    set_page_sub_title(sht, abt_cloud_segment, (lef_tbl_cell[0] + df_cfm_pivot.shape[0] + 2, 1))

    # write table 9, cloud value
    lef_tbl_cell = (lef_tbl_cell[0] + df_cfm_pivot.shape[0] + 1 + 3, 1) 
    write_table_name(sht, (lef_tbl_cell[0]-1, lef_tbl_cell[1]), tbl_name_value)    
    write_table(sht, lef_tbl_cell, df_cloud_cross_tab_cc, "#,0")

    # write table 10, cloud growth
    rght_tbl_cell = (lef_tbl_cell[0], df_cloud_cross_tab_cc.shape[1] + 1 + 3)
    write_table_name(sht, (rght_tbl_cell[0]-1, rght_tbl_cell[1]), tbl_name_grwth)
    write_table(sht, rght_tbl_cell, df_cloud_gwth_cross_tab_cc, ".0%")

    # write table 11, cloud share
    lef_tbl_cell = (lef_tbl_cell[0] + df_cloud_cross_tab_cc.shape[0] + 3, 1)         ### just +3 b/c there is no section label
    write_table_name(sht, (lef_tbl_cell[0]-1, lef_tbl_cell[1]), tbl_name_share)    ### shape[0] is row and shape[1] is column
    write_table(sht, lef_tbl_cell, df_cloud_pct_cross_tab_cc, '.0%')


    print("Finished Market: ", market)
    # except:
        # print('error encountered')
        # wb.sheets[market].delete()
        # break
        # # pass

wb.sheets["Sheet1"].delete()

wb.save(file_path)
wb.close()

Finished Market:  SI
Finished Market:  SD&S
Finished Market:  NEOS
Finished Market:  NC&I
Finished Market:  ITO
Finished Market:  ITC
Finished Market:  IT E&T
Finished Market:  Horizontal BPO
Finished Market:  HIS
Finished Market:  HD&S
Finished Market:  HAM
Finished Market:  CAD
Finished Market:  BC
Finished Market:  AM


#### Write Summary Sheet in the Excel File - By Region

In [17]:
file_path = f"C:/Users/{os.getlogin()}/OneDrive - IDC/Documents/Tracker and Others/Market Forecast/2021H1 Market Forecast/2021H1 cycle market forecast vs last cycle.xlsx"

wb = xw.Book(file_path)
wb.sheets.add("Summary - By Region")
sht = wb.sheets("Summary - By Region")


try:
    ##### all in constant currency #######

    # this cycle - value
    df_pivot_cc = dd.pivot(df_cc)
    df_pivot_cc = df_pivot_cc.groupby(['Region']).sum()
    df_pivot_cc.loc['Total'] = df_pivot_cc.sum()

    # this cycle growth
    df_gwth_pivot_cc = df_pivot_cc.pct_change(axis=1)

    # last cycle - value
    df_last_cycle_pivot_cc = dd.pivot(df_last_cycle_cc)
    df_last_cycle_pivot_cc = df_last_cycle_pivot_cc.groupby(['Region']).sum()
    df_last_cycle_pivot_cc.loc['Total'] = df_last_cycle_pivot_cc.sum()

    # last cycle growth
    df_last_cycle_gwth_pivot_cc = df_last_cycle_pivot_cc.pct_change(axis=1)

    # current vs previous cycle - % diff of value
    df_diff_pct_pivot_cc = (df_pivot_cc - df_last_cycle_pivot_cc)/df_last_cycle_pivot_cc

    # current vs last cycle: growth percent difference
    df_gwth_diff_pivot_cc = df_gwth_pivot_cc - df_last_cycle_gwth_pivot_cc

    # projected/modeled (cfm) value
    df_cfm_pivot = dd.pivot(df_cfm)
    df_cfm_pivot = df_cfm_pivot.groupby(['Region']).sum()
    df_cfm_pivot.loc['Total'] = df_cfm_pivot.sum()

    # projected/modeled (cfm) grwth (%)
    df_cfm_gwth_pivot = df_cfm_pivot.pct_change(axis=1)

    # cloud segment - current cycle
    df_mkt_cc = df_cc
    # not sure if this is the best way but cross tab does won't let me filter df in function 
    # cloud/non-cloud split - value
    df_cloud_cross_tab_cc = pd.crosstab(
        [df_mkt_cc.Region, df_mkt_cc['Delivery Type']], 
        df_mkt_cc.Year, 
        values = df_mkt_cc["Value (Constant)"], 
        aggfunc = sum, 
        margins = False,
        rownames = ['Region', "Cloud/Non-Cloud"], 
        colnames = ['Year'],)
        # normalize = False)    #### don't think we really need this for the sub-total % calculation

    # cloud/non-cloud grwth (%)
    df_cloud_gwth_cross_tab_cc = df_cloud_cross_tab_cc.pct_change(axis=1)

    # cloud/non-cloud share of total (%)
    # calculate the total first
    df_cross_tab_cc = pd.crosstab(
        [df_mkt_cc.Region], 
        df_mkt_cc.Year, 
        values = df_mkt_cc["Value (Constant)"], 
        aggfunc = sum, 
        margins = False,
        rownames = ['Region'], 
        colnames = ['Year'],)
        # normalize = False)    #### don't think we really need this for the sub-total % calculation     
    # then divide cloud/non-cloud by total
    df_cloud_pct_cross_tab_cc = df_cloud_cross_tab_cc / df_cross_tab_cc


    # Writing to excel
    ### - read sheet object

    ### - write label: current cycle
    ### - write table 1 (left), format the table, format header
    ### - write table 2 (right), format the table, format header

    ### - write label: last cycle
    ### - write table 3 (left), format the table, format header
    ### - write table 4 (right), format the table, format header

    ### - write label: current vs last cycle
    ### - write table 5 (left), format the table, format header
    ### - write table 6 (right), format the table, format header

    ### - write label CFM
    ### - write table 7 (left), format the table, format header
    ### - write table 8 (right), format the table, format header


    ### - write label: cloud segmentation

    ### - write table 7 (left), cloud segment - share, format table, format header



    # NOTE: The separate + 1 in the calculation of the range is to account for the header of the table

    # set first tab location
    # position the tables
    lef_tbl_cell = (5, 1)  # set the position of table 1 (left)

    # title - foundation market
    set_page_title(sht, 'All Markets', 'A1')
    set_page_sub_title(sht, "All revenue figures are in CC (Constant Currency) except for CFM Data", 'G1')

    # table 1 & 2 section label: current cycle
    set_page_sub_title(sht, abt_current_cycle, "A3")

    # write table 1
    write_table_name(sht, ((lef_tbl_cell[0]-1), lef_tbl_cell[1]), tbl_name_value)
    write_table(sht, lef_tbl_cell, df_pivot_cc, "#,0")

    # write table 2
    rght_tbl_cell = (lef_tbl_cell[0], df_pivot_cc.shape[1] + 1 + 3)  ### shape[0] is row and shape[1] is column
    write_table_name(sht, ((rght_tbl_cell[0]-1), rght_tbl_cell[1]), tbl_name_grwth)
    write_table(sht, rght_tbl_cell, df_gwth_pivot_cc, ".0%")

    # table 3 & 4 section label: last cycle
    set_page_sub_title(sht, abt_last_cycle, (lef_tbl_cell[0] + df_pivot_cc.shape[0] + 2, 1))

    # write table 3
    lef_tbl_cell = (lef_tbl_cell[0] + df_pivot_cc.shape[0] + 1 + 3, 1)
    write_table_name(sht, ((lef_tbl_cell[0]-1), lef_tbl_cell[1]), tbl_name_value)    
    write_table(sht, lef_tbl_cell, df_last_cycle_pivot_cc, "#,0")

    # write table 4
    rght_tbl_cell = (lef_tbl_cell[0], df_last_cycle_pivot_cc.shape[1] + 1 + 3)  ### shape[0] is row and shape[1] is column
    write_table_name(sht, ((rght_tbl_cell[0]-1), rght_tbl_cell[1]), tbl_name_grwth)
    write_table(sht, rght_tbl_cell, df_last_cycle_gwth_pivot_cc, ".0%")

    # table 5 & 6 section label: current vs last cycle
    set_page_sub_title(sht, abt_current_vs_last, (lef_tbl_cell[0] + df_last_cycle_pivot_cc.shape[0] + 2, 1))

    # write table 5: current vs last % difference    
    lef_tbl_cell = (lef_tbl_cell[0] + df_last_cycle_pivot_cc.shape[0] + 1 + 3, 1)
    write_table_name(sht, ((lef_tbl_cell[0]-1), lef_tbl_cell[1]), tbl_name_diff)
    write_table(sht, lef_tbl_cell, df_diff_pct_pivot_cc, ".0%")

    # write table 6
    rght_tbl_cell = (lef_tbl_cell[0], df_diff_pct_pivot_cc.shape[1] + 1 + 3)        ### shape[0] is row and shape[1] is column
    write_table_name(sht, ((rght_tbl_cell[0]-1), rght_tbl_cell[1]), tbl_name_grwth)
    write_table(sht, rght_tbl_cell, df_gwth_diff_pivot_cc, ".0%")    

    # table 7 & 8 section label: CFM
    set_page_sub_title(sht, abt_CFM, (lef_tbl_cell[0] + df_diff_pct_pivot_cc.shape[0] + 2, 1))

    # write tabe 7: CFM value
    lef_tbl_cell = (lef_tbl_cell[0] + df_diff_pct_pivot_cc.shape[0] + 1 + 3, 1)     ### shape[0] is row and shape[1] is column
    write_table_name(sht, ((lef_tbl_cell[0]-1), lef_tbl_cell[1]), tbl_name_value)    
    write_table(sht, lef_tbl_cell, df_cfm_pivot, "#,0")

    # write table 8
    rght_tbl_cell = (lef_tbl_cell[0], df_cfm_pivot.shape[1] + 1 + 3)     ### shape[0] is row and shape[1] is column
    write_table_name(sht, (rght_tbl_cell[0]-1, rght_tbl_cell[1]), tbl_name_grwth)
    write_table(sht, rght_tbl_cell, df_cfm_gwth_pivot, ".0%")

    # table 9, 10, 11 section label: cloud related
    set_page_sub_title(sht, abt_cloud_segment, (lef_tbl_cell[0] + df_cfm_pivot.shape[0] + 2, 1))

    # write table 9, cloud value
    lef_tbl_cell = (lef_tbl_cell[0] + df_cfm_pivot.shape[0] + 1 + 3, 1) 
    write_table_name(sht, (lef_tbl_cell[0]-1, lef_tbl_cell[1]), tbl_name_value)    
    write_table(sht, lef_tbl_cell, df_cloud_cross_tab_cc, "#,0")

    # write table 10, cloud growth
    rght_tbl_cell = (lef_tbl_cell[0], df_cloud_cross_tab_cc.shape[1] + 1 + 3)
    write_table_name(sht, (rght_tbl_cell[0]-1, rght_tbl_cell[1]), tbl_name_grwth)
    write_table(sht, rght_tbl_cell, df_cloud_gwth_cross_tab_cc, ".0%")

    # write table 11, cloud share
    lef_tbl_cell = (lef_tbl_cell[0] + df_cloud_cross_tab_cc.shape[0] + 3, 1)         ### just +3 b/c there is no section label
    write_table_name(sht, (lef_tbl_cell[0]-1, lef_tbl_cell[1]), tbl_name_share)    ### shape[0] is row and shape[1] is column
    write_table(sht, lef_tbl_cell, df_cloud_pct_cross_tab_cc, '.0%')
    
    print("summary sheet is done!")
except:
    print('error encountered')
    raise

summary sheet is done!


#### Write Summary Sheet in the Excel File - By Market
- Constant Currency

In [18]:
file_path = f"C:/Users/{os.getlogin()}/OneDrive - IDC/Documents/Tracker and Others/Market Forecast/2021H1 Market Forecast/2021H1 cycle market forecast vs last cycle.xlsx"

wb = xw.Book(file_path)
wb.sheets.add("Summary - By Market")
sht = wb.sheets("Summary - By Market")


try:
    ##### all in constant currency #######

    # this cycle - value
    df_pivot_cc = dd.pivot(df_cc)
    df_pivot_cc = df_pivot_cc.groupby(['Foundation Market']).sum()
    df_pivot_cc.loc['Total'] = df_pivot_cc.sum()

    # this cycle growth
    df_gwth_pivot_cc = df_pivot_cc.pct_change(axis=1)

    # last cycle - value
    df_last_cycle_pivot_cc = dd.pivot(df_last_cycle_cc)
    df_last_cycle_pivot_cc = df_last_cycle_pivot_cc.groupby(['Foundation Market']).sum()
    df_last_cycle_pivot_cc.loc['Total'] = df_last_cycle_pivot_cc.sum()

    # last cycle growth
    df_last_cycle_gwth_pivot_cc = df_last_cycle_pivot_cc.pct_change(axis=1)

    # current vs previous cycle - % diff of value
    df_diff_pct_pivot_cc = (df_pivot_cc - df_last_cycle_pivot_cc)/df_last_cycle_pivot_cc

    # current vs last cycle: growth percent difference
    df_gwth_diff_pivot_cc = df_gwth_pivot_cc - df_last_cycle_gwth_pivot_cc

    # projected/modeled (cfm) value
    df_cfm_pivot = dd.pivot(df_cfm)
    df_cfm_pivot = df_cfm_pivot.groupby(['Foundation Market']).sum()
    df_cfm_pivot.loc['Total'] = df_cfm_pivot.sum()

    # projected/modeled (cfm) grwth (%)
    df_cfm_gwth_pivot = df_cfm_pivot.pct_change(axis=1)

    # cloud segment - current cycle
    df_mkt_cc = df_cc
    # not sure if this is the best way but cross tab does won't let me filter df in function 
    # cloud/non-cloud split - value
    df_cloud_cross_tab_cc = pd.crosstab(
        [df_mkt_cc['Foundation Market'], df_mkt_cc['Delivery Type']], 
        df_mkt_cc.Year, 
        values = df_mkt_cc["Value (Constant)"], 
        aggfunc = sum, 
        margins = False,
        rownames = ['Foundation Market', "Cloud/Non-Cloud"], 
        colnames = ['Year'],)
        # normalize = False)    #### don't think we really need this for the sub-total % calculation

    # cloud/non-cloud grwth (%)
    df_cloud_gwth_cross_tab_cc = df_cloud_cross_tab_cc.pct_change(axis=1)

    # cloud/non-cloud share of total (%)
    # calculate the total first
    df_cross_tab_cc = pd.crosstab(
        [df_mkt_cc['Foundation Market']], 
        df_mkt_cc.Year, 
        values = df_mkt_cc["Value (Constant)"], 
        aggfunc = sum, 
        margins = False,
        rownames = ['Foundation Market'], 
        colnames = ['Year'])
        # normalize = False)    #### don't think we really need this for the sub-total % calculation     
    # then divide cloud/non-cloud by total
    df_cloud_pct_cross_tab_cc = df_cloud_cross_tab_cc / df_cross_tab_cc


    # Writing to excel
    ### - read sheet object

    ### - write label: current cycle
    ### - write table 1 (left), format the table, format header
    ### - write table 2 (right), format the table, format header

    ### - write label: last cycle
    ### - write table 3 (left), format the table, format header
    ### - write table 4 (right), format the table, format header

    ### - write label: current vs last cycle
    ### - write table 5 (left), format the table, format header
    ### - write table 6 (right), format the table, format header

    ### - write label CFM
    ### - write table 7 (left), format the table, format header
    ### - write table 8 (right), format the table, format header


    ### - write label: cloud segmentation

    ### - write table 7 (left), cloud segment - share, format table, format header



    # NOTE: The separate + 1 in the calculation of the range is to account for the header of the table

    # set first tab location
    # position the tables
    lef_tbl_cell = (5, 1)  # set the position of table 1 (left)

    # title - foundation market
    set_page_title(sht, 'All Markets', 'A1')
    set_page_sub_title(sht, "All revenue figures are in CC (Constant Currency) except for CFM Data", 'G1')

    # table 1 & 2 section label: current cycle
    set_page_sub_title(sht, abt_current_cycle, "A3")

    # write table 1
    write_table_name(sht, ((lef_tbl_cell[0]-1), lef_tbl_cell[1]), tbl_name_value)
    write_table(sht, lef_tbl_cell, df_pivot_cc, "#,0")

    # write table 2
    rght_tbl_cell = (lef_tbl_cell[0], df_pivot_cc.shape[1] + 1 + 3)  ### shape[0] is row and shape[1] is column
    write_table_name(sht, ((rght_tbl_cell[0]-1), rght_tbl_cell[1]), tbl_name_grwth)
    write_table(sht, rght_tbl_cell, df_gwth_pivot_cc, ".0%")

    # table 3 & 4 section label: last cycle
    set_page_sub_title(sht, abt_last_cycle, (lef_tbl_cell[0] + df_pivot_cc.shape[0] + 2, 1))

    # write table 3
    lef_tbl_cell = (lef_tbl_cell[0] + df_pivot_cc.shape[0] + 1 + 3, 1)
    write_table_name(sht, ((lef_tbl_cell[0]-1), lef_tbl_cell[1]), tbl_name_value)    
    write_table(sht, lef_tbl_cell, df_last_cycle_pivot_cc, "#,0")

    # write table 4
    rght_tbl_cell = (lef_tbl_cell[0], df_last_cycle_pivot_cc.shape[1] + 1 + 3)  ### shape[0] is row and shape[1] is column
    write_table_name(sht, ((rght_tbl_cell[0]-1), rght_tbl_cell[1]), tbl_name_grwth)
    write_table(sht, rght_tbl_cell, df_last_cycle_gwth_pivot_cc, ".0%")

    # table 5 & 6 section label: current vs last cycle
    set_page_sub_title(sht, abt_current_vs_last, (lef_tbl_cell[0] + df_last_cycle_pivot_cc.shape[0] + 2, 1))

    # write table 5: current vs last % difference    
    lef_tbl_cell = (lef_tbl_cell[0] + df_last_cycle_pivot_cc.shape[0] + 1 + 3, 1)
    write_table_name(sht, ((lef_tbl_cell[0]-1), lef_tbl_cell[1]), tbl_name_diff)
    write_table(sht, lef_tbl_cell, df_diff_pct_pivot_cc, ".0%")

    # write table 6
    rght_tbl_cell = (lef_tbl_cell[0], df_diff_pct_pivot_cc.shape[1] + 1 + 3)        ### shape[0] is row and shape[1] is column
    write_table_name(sht, ((rght_tbl_cell[0]-1), rght_tbl_cell[1]), tbl_name_grwth)
    write_table(sht, rght_tbl_cell, df_gwth_diff_pivot_cc, ".0%")    

    #### deleted by accident need to fix!

summary sheet is done!


#### Write Release Notes Related Sheet in the Excel File

In [19]:
wb.sheets.add("for release notes")
sht = wb.sheets("for release notes")

prime_mkts = df_cc['Primary Market'].unique()
lef_tbl_cell = (6, 1)  # set the position of table 1 (left)

# title for the page
set_page_title(sht, 'By Markets - for Release Notes', 'A1')
set_page_sub_title(sht, "All revenue figures are in CC (Constant Currency) except for CFM Data", 'G1')

#### loop throuhg primary markets
for mkt in prime_mkts:
    # this cycle - value
    df_pivot_cc = dd.pivot(df_cc[df_cc["Primary Market"]==mkt])
    df_pivot_cc = df_pivot_cc.groupby(['Foundation Market']).sum()
    df_pivot_cc.loc['Total'] = df_pivot_cc.sum()

    # this cycle growth
    df_gwth_pivot_cc = df_pivot_cc.pct_change(axis=1)

    # last cycle - value
    df_last_cycle_pivot_cc = dd.pivot(df_last_cycle_cc[df_last_cycle_cc["Primary Market"]==mkt])
    df_last_cycle_pivot_cc = df_last_cycle_pivot_cc.groupby(['Foundation Market']).sum()
    df_last_cycle_pivot_cc.loc['Total'] = df_last_cycle_pivot_cc.sum()

    # last cycle growth
    df_last_cycle_gwth_pivot_cc = df_last_cycle_pivot_cc.pct_change(axis=1)

    # current vs previous cycle - % diff of value
    df_diff_pct_pivot_cc = (df_pivot_cc - df_last_cycle_pivot_cc)/df_last_cycle_pivot_cc

    # current vs last cycle: growth percent difference
    df_gwth_diff_pivot_cc = df_gwth_pivot_cc - df_last_cycle_gwth_pivot_cc

    ### write to sheet
    set_page_title(sht, mkt, (lef_tbl_cell[0]-3, 1))

    # table 1 & 2 section label: current cycle
    set_page_sub_title(sht, abt_current_cycle, (lef_tbl_cell[0]-2, 1))

    # write table 1
    write_table_name(sht, ((lef_tbl_cell[0]-1), lef_tbl_cell[1]), tbl_name_value)
    write_table(sht, lef_tbl_cell, df_pivot_cc, "#,0")

    # write table 2
    rght_tbl_cell = (lef_tbl_cell[0], df_pivot_cc.shape[1] + 1 + 3)  ### shape[0] is row and shape[1] is column
    write_table_name(sht, ((rght_tbl_cell[0]-1), rght_tbl_cell[1]), tbl_name_grwth)
    write_table(sht, rght_tbl_cell, df_gwth_pivot_cc, ".0%")

    # table 3 & 4 section label: last cycle
    set_page_sub_title(sht, abt_last_cycle, (lef_tbl_cell[0] + df_pivot_cc.shape[0] + 2, 1))

    # write table 3
    lef_tbl_cell = (lef_tbl_cell[0] + df_pivot_cc.shape[0] + 1 + 3, 1)
    write_table_name(sht, ((lef_tbl_cell[0]-1), lef_tbl_cell[1]), tbl_name_value)    
    write_table(sht, lef_tbl_cell, df_last_cycle_pivot_cc, "#,0")

    # write table 4
    rght_tbl_cell = (lef_tbl_cell[0], df_last_cycle_pivot_cc.shape[1] + 1 + 3)  ### shape[0] is row and shape[1] is column
    write_table_name(sht, ((rght_tbl_cell[0]-1), rght_tbl_cell[1]), tbl_name_grwth)
    write_table(sht, rght_tbl_cell, df_last_cycle_gwth_pivot_cc, ".0%")

    # table 5 & 6 section label: current vs last cycle
    set_page_sub_title(sht, abt_current_vs_last, (lef_tbl_cell[0] + df_last_cycle_pivot_cc.shape[0] + 2, 1))

    # write table 5: current vs last % difference    
    lef_tbl_cell = (lef_tbl_cell[0] + df_last_cycle_pivot_cc.shape[0] + 1 + 3, 1)
    write_table_name(sht, ((lef_tbl_cell[0]-1), lef_tbl_cell[1]), tbl_name_diff)
    write_table(sht, lef_tbl_cell, df_diff_pct_pivot_cc, ".0%")

    # write table 6
    rght_tbl_cell = (lef_tbl_cell[0], df_diff_pct_pivot_cc.shape[1] + 1 + 3)       ### shape[0] is row and shape[1] is column
    write_table_name(sht, ((rght_tbl_cell[0]-1), rght_tbl_cell[1]), tbl_name_grwth)
    write_table(sht, rght_tbl_cell, df_gwth_diff_pivot_cc, ".0%")
    
    # shift cell position down by 1 row
    lef_tbl_cell = (lef_tbl_cell[0] + df_last_cycle_pivot_cc.shape[0] + 1 + 3 + 2, 1)
    
print('Summary sheet done!')

Summary sheet done!


#### Write Managed Cloud Sheet
- @ cc

In [13]:
file_path = f"C:/Users/{os.getlogin()}/OneDrive - IDC/Documents/Tracker and Others/Market Forecast/2021H1 Market Forecast/2021H1 cycle market forecast vs last cycle.xlsx"
sheet_title = ("Managed Cloud Summary")

wb = xw.Book(file_path)
wb.sheets.add(sheet_title)
sht = wb.sheets(sheet_title)

mgt_cloud_mkts = ['AM', 'HAM', 'HIS', 'ITO', 'NEOS']
lef_tbl_cell = (5, 1)  # set the position of table 1 (left)

# try:
##### all in constant currency #######

# this cycle - value
df_pivot_cc = dd.pivot(df_cc[df_cc['Foundation Market'].isin(mgt_cloud_mkts)])
df_pivot_cc = df_pivot_cc.groupby(['Region']).sum()
df_pivot_cc.loc['Total'] = df_pivot_cc.sum()

# this cycle growth
df_gwth_pivot_cc = df_pivot_cc.pct_change(axis=1)

# cloud segment - current cycle
df_mkt_cc = df_cc[df_cc['Foundation Market'].isin(mgt_cloud_mkts)]
# not sure if this is the best way but cross tab does won't let me filter df in function 
# cloud/non-cloud split - value
df_cloud_cross_tab_cc = pd.crosstab(
    [df_mkt_cc['Region'], df_mkt_cc['Delivery Type']], 
    df_mkt_cc.Year, 
    values = df_mkt_cc["Value (Constant)"], 
    aggfunc = sum, 
    margins = False,
    rownames = ['Region', "Cloud/Non-Cloud"], 
    colnames = ['Year'],)
    # normalize = False)    #### don't think we really need this for the sub-total % calculation

# cloud/non-cloud grwth (%)
df_cloud_gwth_cross_tab_cc = df_cloud_cross_tab_cc.pct_change(axis=1)

# cloud/non-cloud share of total (%)
# calculate the total first
df_cross_tab_cc = pd.crosstab(
    [df_mkt_cc['Region']], 
    df_mkt_cc.Year, 
    values = df_mkt_cc["Value (Constant)"], 
    aggfunc = sum, 
    margins = False,
    rownames = ['Region'], 
    colnames = ['Year'])
    # normalize = False)    #### don't think we really need this for the sub-total % calculation     
# then divide cloud/non-cloud by total
df_cloud_pct_cross_tab_cc = df_cloud_cross_tab_cc / df_cross_tab_cc


# Writing to excel
### - read sheet object

### - write label: current cycle
### - write table 1 (left), format the table, format header
### - write table 2 (right), format the table, format header

### - write label: cloud segmentation
### - write table 7 (left), cloud segment - share, format table, format header



# NOTE: The separate + 1 in the calculation of the range is to account for the header of the table

# set first tab location
# position the tables
lef_tbl_cell = (5, 1)  # set the position of table 1 (left)

# title - foundation market
set_page_title(sht, 'Mangaged Cloud Related Markets Only: AM, HAM, HIS, ITO, NEO', 'A1')
set_page_sub_title(sht, "All revenue figures are in CC (Constant Currency)", 'G1')

# table 1 & 2 section label: current cycle
set_page_sub_title(sht, abt_current_cycle, "A3")

# write table 1
write_table_name(sht, ((lef_tbl_cell[0]-1), lef_tbl_cell[1]), tbl_name_value)
write_table(sht, lef_tbl_cell, df_pivot_cc, "#,0")

# write table 2
rght_tbl_cell = (lef_tbl_cell[0], df_pivot_cc.shape[1] + 1 + 3)  ### shape[0] is row and shape[1] is column
write_table_name(sht, ((rght_tbl_cell[0]-1), rght_tbl_cell[1]), tbl_name_grwth)
write_table(sht, rght_tbl_cell, df_gwth_pivot_cc, ".0%")

# table 3, 4, 5 section label: cloud related
set_page_sub_title(sht, abt_cloud_segment, (lef_tbl_cell[0] + df_pivot_cc.shape[0] + 2, 1))

# write table 9, cloud value
lef_tbl_cell = (lef_tbl_cell[0] + df_pivot_cc.shape[0] + 1 + 3, 1) 
write_table_name(sht, (lef_tbl_cell[0]-1, lef_tbl_cell[1]), tbl_name_value)    
write_table(sht, lef_tbl_cell, df_cloud_cross_tab_cc, "#,0")

# write table 10, cloud growth
rght_tbl_cell = (lef_tbl_cell[0], df_cloud_cross_tab_cc.shape[1] + 1 + 3)
write_table_name(sht, (rght_tbl_cell[0]-1, rght_tbl_cell[1]), tbl_name_grwth)
write_table(sht, rght_tbl_cell, df_cloud_gwth_cross_tab_cc, ".0%")

# write table 11, cloud share
lef_tbl_cell = (lef_tbl_cell[0] + df_cloud_cross_tab_cc.shape[0] + 3, 1)         ### just +3 b/c there is no section label
write_table_name(sht, (lef_tbl_cell[0]-1, lef_tbl_cell[1]), tbl_name_share)    ### shape[0] is row and shape[1] is column
write_table(sht, lef_tbl_cell, df_cloud_pct_cross_tab_cc, '.0%')

print("mgt cloud summary table - done!")

mgt cloud summary table - done!


# Data Analysis

## Execute Pivot Functions & Create Parameters

In [21]:
#### run all the codes to get the crosstabs and pivots

# df_pivot_cc = dd.pivot(df_cc)
# df_pivot_cc = df_pivot_cc.groupby(['Region']).sum()
# df_pivot_cc.loc['Total'] = df_pivot_cc.sum()
# df_gwth_pivot_cc = df_pivot_cc.pct_change(axis=1)

# df_last_cycle_pivot_cc = dd.pivot(df_last_cycle_cc)
# df_last_cycle_pivot_cc = df_last_cycle_pivot_cc.groupby(['Region']).sum()
# df_last_cycle_pivot_cc.loc['Total'] = df_last_cycle_pivot_cc.sum()
# df_last_cycle_gwth_pivot_cc = df_last_cycle_pivot_cc.pct_change(axis=1)

# df_diff_pct_pivot_cc = (df_pivot_cc - df_last_cycle_pivot_cc)/df_last_cycle_pivot_cc
# df_gwth_diff_pivot_cc = df_gwth_pivot_cc - df_last_cycle_gwth_pivot_cc

# df_cfm_pivot = dd.pivot(df_cfm)
# df_cfm_pivot = df_cfm_pivot.groupby(['Region']).sum()
# df_cfm_pivot.loc['Total'] = df_cfm_pivot.sum()
# df_cfm_gwth_pivot = df_cfm_pivot.pct_change(axis=1)
# df_mkt_cc = df_cc
# df_cloud_cross_tab_cc = pd.crosstab(
#     [df_mkt_cc.Region, df_mkt_cc['Delivery Type']], 
#     df_mkt_cc.Year, 
#     values = df_mkt_cc["Value (Constant)"], 
#     aggfunc = sum, 
#     margins = False,
#     rownames = ['Region', "Cloud/Non-Cloud"], 
#     colnames = ['Year'],)
#     # normalize = False)    #### don't think we really need this for the sub-total % calculation
# df_cloud_gwth_cross_tab_cc = df_cloud_cross_tab_cc.pct_change(axis=1)
# df_cross_tab_cc = pd.crosstab(
#     [df_mkt_cc.Region], 
#     df_mkt_cc.Year, 
#     values = df_mkt_cc["Value (Constant)"], 
#     aggfunc = sum, 
#     margins = False,
#     rownames = ['Region'], 
#     colnames = ['Year'],)
#     # normalize = False)    #### don't think we really need this for the sub-total % calculation     
# df_cloud_pct_cross_tab_cc = df_cloud_cross_tab_cc / df_cross_tab_cc

In [25]:
region_filter = ['APeJC', 'JP', "PRC"]
groupby_geo = ['Country']

df_pivot_cc = dd.pivot(df_cc[df_cc.Region.isin(region_filter)])
df_pivot_cc = df_pivot_cc.groupby(groupby_geo).sum()
df_pivot_cc.loc['Total'] = df_pivot_cc.sum()
df_gwth_pivot_cc = df_pivot_cc.pct_change(axis=1)

df_last_cycle_pivot_cc = dd.pivot(df_last_cycle_cc[df_last_cycle_cc.Region.isin(region_filter)])
df_last_cycle_pivot_cc = df_last_cycle_pivot_cc.groupby(groupby_geo).sum()
df_last_cycle_pivot_cc.loc['Total'] = df_last_cycle_pivot_cc.sum()
df_last_cycle_gwth_pivot_cc = df_last_cycle_pivot_cc.pct_change(axis=1)

df_diff_pct_pivot_cc = (df_pivot_cc - df_last_cycle_pivot_cc)/df_last_cycle_pivot_cc
df_gwth_diff_pivot_cc = df_gwth_pivot_cc - df_last_cycle_gwth_pivot_cc

pd.set_option('display.float_format', '{:,.0f}'.format)
display("This cycle:", df_pivot_cc, "Last Cycle:", df_last_cycle_pivot_cc)
pd.set_option('display.float_format', '{:,.2%}'.format)
display("Difference:", df_gwth_diff_pivot_cc)

# pd.set_option('display.float.format', '{:,.0f}'.format)
# display('Differences', df_pivot_cc - df_last_cycle_pivot_cc)

# pd.set_option('display.float_format', '{:,.2%}'.format)
display("this cycle growth:", df_gwth_pivot_cc, 'last cycle growth', df_last_cycle_gwth_pivot_cc, 'growth difference', df_gwth_diff_pivot_cc)

'This cycle:'

,2018,2019,2020,2021,2022,2023,2024,2025
Country,,,,,,,,
Australia,"24,489","25,587","26,156","26,984","28,004","28,826","29,527","30,116"
Hong Kong,"3,262","3,397","3,472","3,620","3,784","3,953","4,125","4,299"
India,"11,747","12,738","13,415","14,311","15,420","16,735","18,228","19,935"
Indonesia,"1,690","1,944","2,098","2,310","2,584","2,926","3,348","3,870"
Japan,"62,881","65,333","64,361","66,987","69,252","71,425","73,554","75,680"
Korea,"10,030","10,378","10,424","10,573","10,789","10,983","11,159","11,315"
Malaysia,"3,975","4,205","4,301","4,517","4,762","5,031","5,327","5,653"
New Zealand,"3,599","3,733","3,854","3,987","4,131","4,263","4,386","4,504"
PRC,"39,246","42,319","43,168","47,835","49,767","53,575","57,202","61,214"


'Last Cycle:'

,2018,2019,2020,2021,2022,2023,2024,2025
Country,,,,,,,,
Australia,"23,263","24,306","24,847","25,744","26,512","27,172","27,758","28,255"
Hong Kong,"3,223","3,355","3,427","3,573","3,731","3,892","4,056","4,222"
India,"11,734","12,723","13,411","14,260","15,233","16,332","17,570","18,969"
Indonesia,"1,697","1,953","2,107","2,322","2,602","2,938","3,348","3,856"
Japan,"65,916","68,487","67,468","70,274","72,651","74,929","77,163","79,392"
Korea,"9,981","10,328","10,374","10,522","10,740","10,935","11,104","11,245"
Malaysia,"3,959","4,187","4,282","4,504","4,754","5,029","5,331","5,664"
New Zealand,"3,456","3,584","3,700","3,833","3,965","4,085","4,195","4,296"
PRC,"36,863","39,659","40,356","43,630","47,227","50,870","54,607","58,743"


'Difference:'

...

,2018,2019,2020,2021,2022,2023,2024,2025
Country,,,,,,,,
Australia,NaN,4.48%,2.23%,3.16%,3.78%,2.93%,2.43%,1.99%
Hong Kong,NaN,4.16%,2.19%,4.28%,4.52%,4.46%,4.35%,4.23%
India,NaN,8.43%,5.31%,6.68%,7.75%,8.53%,8.92%,9.36%
Indonesia,NaN,15.07%,7.88%,10.12%,11.87%,13.23%,14.42%,15.59%
Japan,NaN,3.90%,-1.49%,4.08%,3.38%,3.14%,2.98%,2.89%
Korea,NaN,3.47%,0.44%,1.43%,2.04%,1.80%,1.60%,1.41%
Malaysia,NaN,5.80%,2.29%,5.02%,5.41%,5.66%,5.88%,6.11%
New Zealand,NaN,3.71%,3.24%,3.46%,3.60%,3.19%,2.88%,2.70%
PRC,NaN,7.83%,2.01%,10.81%,4.04%,7.65%,6.77%,7.01%


'last cycle growth'

,2018,2019,2020,2021,2022,2023,2024,2025
Country,,,,,,,,
Australia,NaN,4.48%,2.23%,3.61%,2.98%,2.49%,2.15%,1.79%
Hong Kong,NaN,4.10%,2.14%,4.27%,4.41%,4.31%,4.22%,4.10%
India,NaN,8.43%,5.41%,6.33%,6.82%,7.22%,7.58%,7.96%
Indonesia,NaN,15.07%,7.88%,10.20%,12.07%,12.90%,13.97%,15.15%
Japan,NaN,3.90%,-1.49%,4.16%,3.38%,3.13%,2.98%,2.89%
Korea,NaN,3.47%,0.44%,1.43%,2.07%,1.81%,1.54%,1.27%
Malaysia,NaN,5.77%,2.26%,5.19%,5.55%,5.78%,6.00%,6.25%
New Zealand,NaN,3.71%,3.24%,3.60%,3.43%,3.03%,2.70%,2.41%
PRC,NaN,7.58%,1.76%,8.11%,8.24%,7.71%,7.35%,7.57%


'growth difference'

,2018,2019,2020,2021,2022,2023,2024,2025
Country,,,,,,,,
Australia,NaN,0.00%,-0.00%,-0.45%,0.80%,0.44%,0.28%,0.20%
Hong Kong,NaN,0.06%,0.06%,0.01%,0.11%,0.15%,0.12%,0.13%
India,NaN,0.00%,-0.09%,0.35%,0.93%,1.31%,1.34%,1.40%
Indonesia,NaN,0.00%,-0.00%,-0.08%,-0.20%,0.33%,0.46%,0.43%
Japan,NaN,-0.00%,-0.00%,-0.08%,-0.00%,0.00%,-0.00%,0.00%
Korea,NaN,-0.01%,0.00%,0.00%,-0.04%,-0.01%,0.06%,0.13%
Malaysia,NaN,0.03%,0.03%,-0.17%,-0.14%,-0.12%,-0.12%,-0.14%
New Zealand,NaN,0.00%,0.00%,-0.14%,0.17%,0.17%,0.18%,0.29%
PRC,NaN,0.24%,0.25%,2.70%,-4.21%,-0.06%,-0.58%,-0.56%


In [98]:
df_cc['Primary Market'].unique()

array(['Managed Services', 'Support Services', 'Project Oriented'],
      dtype=object)

In [108]:
# the prim to foun market part

# Project Oriented Services
# Managed Services
# Support Services


# market_filter = ['Project Oriented'] 
# market_filter = ['Managed Services'] 
# market_filter = ['Support Services']
market_filter = ['Project Oriented', 'Managed Services', "Support Services"]
groupby_mkt = ['Foundation Market']

df_pivot_cc = dd.pivot(df_cc[df_cc['Primary Market'].isin(market_filter)])
df_pivot_cc = df_pivot_cc.groupby(groupby_mkt).sum()
df_pivot_cc.loc['Total'] = df_pivot_cc.sum()
df_gwth_pivot_cc = df_pivot_cc.pct_change(axis=1)

df_last_cycle_pivot_cc = dd.pivot(df_last_cycle_cc[df_last_cycle_cc['Primary Market'].isin(market_filter)])
df_last_cycle_pivot_cc = df_last_cycle_pivot_cc.groupby(groupby_mkt).sum()
df_last_cycle_pivot_cc.loc['Total'] = df_last_cycle_pivot_cc.sum()
df_last_cycle_gwth_pivot_cc = df_last_cycle_pivot_cc.pct_change(axis=1)

df_diff_pct_pivot_cc = (df_pivot_cc - df_last_cycle_pivot_cc)/df_last_cycle_pivot_cc
df_gwth_diff_pivot_cc = df_gwth_pivot_cc - df_last_cycle_gwth_pivot_cc

pd.set_option('display.float_format', '{:,.0f}'.format)
display("This cycle:", df_pivot_cc, "Last Cycle:", df_last_cycle_pivot_cc)
pd.set_option('display.float_format', '{:,.2%}'.format)
display("Difference:", df_gwth_diff_pivot_cc)

# pd.set_option('display.float.format', '{:,.0f}'.format)
# display('Differences', df_pivot_cc - df_last_cycle_pivot_cc)

# pd.set_option('display.float_format', '{:,.2%}'.format)
display("this cycle growth:", df_gwth_pivot_cc, 'last cycle growth', df_last_cycle_gwth_pivot_cc, 'growth difference', df_gwth_diff_pivot_cc)

'This cycle:'

,2018,2019,2020,2021,2022,2023,2024,2025
Foundation Market,,,,,,,,
AM,"68,756","73,541","73,556","76,172","78,978","81,840","84,638","87,457"
BC,"125,286","135,631","129,277","137,013","146,158","156,402","167,443","179,003"
CAD,"47,246","51,083","49,927","52,689","55,455","58,631","61,919","65,166"
HAM,"14,453","15,856","16,630","17,723","18,822","19,971","21,135","22,341"
HD&S,"64,079","65,830","65,942","67,109","68,985","71,103","73,047","75,965"
HIS,"53,828","57,345","60,019","64,339","68,989","73,821","78,841","84,149"
Horizontal BPO,"197,786","207,312","208,174","214,545","221,596","229,917","238,866","248,766"
IT E&T,"18,543","19,373","18,830","20,011","21,034","22,152","23,312","24,530"
ITC,"39,456","41,755","40,830","42,324","44,183","46,319","48,528","50,816"


'Last Cycle:'

,2018,2019,2020,2021,2022,2023,2024,2025
Foundation Market,,,,,,,,
AM,"67,367","72,064","72,072","74,679","77,378","80,018","82,608","85,164"
BC,"123,981","134,304","128,102","135,517","144,331","154,267","164,957","176,289"
CAD,"46,722","50,532","49,373","51,581","54,223","56,927","59,727","62,419"
HAM,"14,222","15,605","16,392","17,472","18,569","19,664","20,770","21,920"
HD&S,"63,426","65,144","65,232","66,449","68,307","70,313","72,193","75,030"
HIS,"51,906","55,217","57,774","61,832","66,085","70,516","75,196","80,254"
Horizontal BPO,"194,556","203,531","204,128","210,561","218,107","226,373","235,154","244,745"
IT E&T,"18,530","19,372","18,845","19,930","20,921","21,949","23,018","24,155"
ITC,"38,853","41,103","40,161","41,527","43,260","45,191","47,185","49,255"


'Difference:'

...

,2018,2019,2020,2021,2022,2023,2024,2025
Foundation Market,,,,,,,,
AM,NaN,6.96%,0.02%,3.56%,3.68%,3.62%,3.42%,3.33%
BC,NaN,8.26%,-4.68%,5.98%,6.67%,7.01%,7.06%,6.90%
CAD,NaN,8.12%,-2.26%,5.53%,5.25%,5.73%,5.61%,5.24%
HAM,NaN,9.71%,4.88%,6.57%,6.20%,6.10%,5.83%,5.70%
HD&S,NaN,2.73%,0.17%,1.77%,2.80%,3.07%,2.73%,3.99%
HIS,NaN,6.53%,4.66%,7.20%,7.23%,7.00%,6.80%,6.73%
Horizontal BPO,NaN,4.82%,0.42%,3.06%,3.29%,3.76%,3.89%,4.14%
IT E&T,NaN,4.48%,-2.80%,6.27%,5.11%,5.32%,5.24%,5.23%
ITC,NaN,5.83%,-2.22%,3.66%,4.39%,4.83%,4.77%,4.71%


'last cycle growth'

,2018,2019,2020,2021,2022,2023,2024,2025
Foundation Market,,,,,,,,
AM,NaN,6.97%,0.01%,3.62%,3.61%,3.41%,3.24%,3.09%
BC,NaN,8.33%,-4.62%,5.79%,6.50%,6.88%,6.93%,6.87%
CAD,NaN,8.15%,-2.29%,4.47%,5.12%,4.99%,4.92%,4.51%
HAM,NaN,9.72%,5.05%,6.59%,6.28%,5.90%,5.62%,5.54%
HD&S,NaN,2.71%,0.13%,1.87%,2.80%,2.94%,2.67%,3.93%
HIS,NaN,6.38%,4.63%,7.02%,6.88%,6.71%,6.64%,6.73%
Horizontal BPO,NaN,4.61%,0.29%,3.15%,3.58%,3.79%,3.88%,4.08%
IT E&T,NaN,4.54%,-2.72%,5.76%,4.97%,4.91%,4.87%,4.94%
ITC,NaN,5.79%,-2.29%,3.40%,4.17%,4.46%,4.41%,4.39%


'growth difference'

,2018,2019,2020,2021,2022,2023,2024,2025
Foundation Market,,,,,,,,
AM,NaN,-0.01%,0.01%,-0.06%,0.07%,0.21%,0.18%,0.24%
BC,NaN,-0.07%,-0.07%,0.20%,0.17%,0.13%,0.13%,0.03%
CAD,NaN,-0.03%,0.03%,1.06%,0.13%,0.74%,0.69%,0.74%
HAM,NaN,-0.01%,-0.16%,-0.02%,-0.08%,0.21%,0.20%,0.17%
HD&S,NaN,0.02%,0.03%,-0.10%,0.00%,0.13%,0.06%,0.06%
HIS,NaN,0.16%,0.03%,0.17%,0.35%,0.30%,0.16%,0.01%
Horizontal BPO,NaN,0.20%,0.12%,-0.09%,-0.30%,-0.03%,0.01%,0.07%
IT E&T,NaN,-0.07%,-0.08%,0.51%,0.14%,0.41%,0.37%,0.29%
ITC,NaN,0.04%,0.08%,0.26%,0.22%,0.37%,0.36%,0.33%


In [106]:
wb = xw.Book()
sht = xw.sheets('Sheet1')
sht.range('A1').value = df_gwth_diff_pivot_cc

In [22]:
df_pivot_cc = dd.pivot(df_cc)
df_pivot_cc = df_pivot_cc.groupby('Global Region').sum()
df_gwth_pivot_cc = df_pivot_cc.pct_change(axis=1)

                            

KeyError: 'Global Region'

In [74]:
pd.set_option('display.float.format', '{:,.2%}'.format)
df_mkt_cc = df_cc[df_cc.Region.isin(region_filter)]

pd.crosstab(
    df_mkt_cc.Country,
    df_mkt_cc.Year, 
    values = df_mkt_cc["Value (Constant)"], 
    aggfunc = sum, 
    margins = False,
    rownames = ['Country'], 
    colnames = ['Year'], normalize='columns')

df_gwth_pivot_cc

,2018,2019,2020,2021,2022,2023,2024,2025
Country,,,,,,,,
Japan,NaN,3.90%,-1.49%,4.08%,3.38%,3.14%,2.98%,2.89%
PRC,NaN,7.83%,2.01%,10.81%,4.04%,7.65%,6.77%,7.01%
Total,NaN,5.41%,-0.11%,6.78%,3.66%,5.03%,4.61%,4.69%


In [33]:
df_last_cycle_cc[df_last_cycle_cc['Foundation Market']=='PEOTS']

,Region,Country,Primary Market,Secondary Market,Foundation Market,Sub-Foundation Market,Delivery Type,Year,Value (Constant)


## Compare Current vs Previous Version

In [23]:
df_cloud_cross_tab_cc = pd.crosstab(
    [df_cc['Foundation Market'], df_cc['Delivery Type']], 
    df_cc.Year, 
    values = df_mkt_cc["Value (Constant)"], 
    aggfunc = sum, 
    margins = False,
    rownames = ['Foundation Market', "Cloud/Non-Cloud"], 
    colnames = ['Year'],)

df_last_cycle_cloud_cross_tab_cc = pd.crosstab(
    [df_last_cycle_cc['Foundation Market'], df_last_cycle_cc['Delivery Type']], 
    df_last_cycle_cc.Year, 
    values = df_last_cycle_cc["Value (Constant)"], 
    aggfunc = sum, 
    margins = False,
    rownames = ['Foundation Market', "Cloud/Non-Cloud"], 
    colnames = ['Year'],)

display(df_cloud_cross_tab_cc)
display(df_last_cycle_cloud_cross_tab_cc)
display((df_cloud_cross_tab_cc - df_last_cycle_cloud_cross_tab_cc)/df_last_cycle_cross_tab_cc)

Year                                    2018       2019       2020       2021  \
Foundation Market Cloud/Non-Cloud                                               
AM                Cloud-Related     8,626.59  10,231.43  13,586.60  16,463.49   
                  non-Cloud        60,129.87  63,310.00  59,969.51  59,708.81   
BC                Cloud-Related    16,351.12  19,683.25  20,471.25  23,548.06   
                  non-Cloud       108,935.32 115,947.40 108,805.55 113,464.90   
CAD               Cloud-Related     6,871.07   8,322.30  10,406.53  12,332.17   
                  non-Cloud        40,374.49  42,760.66  39,520.04  40,356.72   
HAM               Cloud-Related     3,445.63   4,082.21   5,095.39   6,040.62   
                  non-Cloud        11,007.07  11,773.93  11,535.11  11,682.78   
HD&S              Cloud-Related    11,170.47  12,292.68  12,965.11  14,401.51   
                  non-Cloud        52,908.28  53,537.74  52,976.85  52,707.07   
HIS               Cloud-Related    10,513.08  12,601.65  14,996.76  17,982.62   
                  non-Cloud        43,315.13  44,743.62  45,022.21  46,356.61   
Horizontal BPO    Cloud-Related    11,800.32  15,046.49  20,085.28  24,192.13   
                  non-Cloud       185,985.53 192,265.51 188,088.28 190,352.80   
IT E&T            Cloud-Related     5,681.26   6,545.27   7,699.97   8,930.56   
                  non-Cloud        12,861.34  12,827.61  11,130.03  11,080.86   
ITC               Cloud-Related     7,004.84   8,543.88   9,639.79  11,262.02   
                  non-Cloud        32,450.67  33,211.41  31,190.61  31,062.36   
ITO               Cloud-Related    21,226.95  25,138.74  27,950.35  32,796.46   
                  non-Cloud        78,369.14  74,947.61  68,770.14  63,508.72   
NC&I              Cloud-Related     5,378.20   6,308.84   7,287.70   8,605.78   
                  non-Cloud        40,412.17  41,892.85  42,785.50  42,143.15   
NEOS              Cloud-Related     4,572.50   6,173.17   7,805.37  10,205.89   
                  non-Cloud        48,793.33  49,862.30  48,055.22  47,495.94   
SD&S              Cloud-Related     8,590.08  10,049.92  11,502.00  13,745.03   
                  non-Cloud        66,653.31  67,763.44  66,237.00  65,155.81   
SI                Cloud-Related    22,726.58  27,151.11  29,840.11  34,865.81   
                  non-Cloud       111,130.80 112,834.49 104,076.29 103,415.98   

Year                                    2022       2023       2024       2025  
Foundation Market Cloud/Non-Cloud                                              
AM                Cloud-Related    19,732.08  23,378.27  27,039.82  30,597.77  
                  non-Cloud        59,246.18  58,462.08  57,598.33  56,859.46  
BC                Cloud-Related    27,473.71  32,108.54  37,668.97  43,837.66  
                  non-Cloud       118,684.73 124,293.91 129,774.29 135,165.57  
CAD               Cloud-Related    14,593.89  17,080.22  19,732.36  22,639.46  
                  non-Cloud        40,861.32  41,551.12  42,187.06  42,526.30  
HAM               Cloud-Related     7,061.07   8,085.75   9,165.87  10,254.07  
                  non-Cloud        11,761.17  11,885.57  11,969.31  12,086.58  
HD&S              Cloud-Related    15,973.38  17,653.12  19,620.69  21,874.98  
                  non-Cloud        53,011.95  53,449.68  53,426.31  54,090.13  
HIS               Cloud-Related    21,704.14  25,739.03  30,031.09  34,740.85  
                  non-Cloud        47,284.62  48,082.37  48,810.34  49,407.89  
Horizontal BPO    Cloud-Related    27,931.13  32,969.41  37,782.69  43,500.47  
                  non-Cloud       193,664.42 196,947.89 201,083.73 205,265.10  
IT E&T            Cloud-Related    10,169.63  11,538.15  12,856.56  14,097.83  
                  non-Cloud        10,863.95  10,613.88  10,455.63  10,432.56  
ITC               Cloud-Related    13,259.24  15,496.14  17,773.32  20,194.75  
                  non-Cloud        30,924.03  30,

Year                                    2018       2019       2020       2021  \
Foundation Market Cloud/Non-Cloud                                               
AM                Cloud-Related     8,434.62  10,004.16  13,443.76  16,343.29   
                  non-Cloud        58,932.66  62,060.13  58,628.56  58,335.55   
BC                Cloud-Related    30,678.21  35,781.42  42,288.22  49,155.60   
                  non-Cloud        93,303.24  98,522.66  85,813.95  86,361.72   
CAD               Cloud-Related     6,786.76   8,214.98  10,286.65  12,136.32   
                  non-Cloud        39,935.54  42,316.56  39,086.75  39,444.53   
HAM               Cloud-Related     3,389.96   4,014.41   5,047.86   5,978.46   
                  non-Cloud        10,831.92  11,590.32  11,344.34  11,493.97   
HD&S              Cloud-Related    11,026.06  12,130.03  12,790.70  14,236.94   
                  non-Cloud        52,400.34  53,014.45  52,441.67  52,212.19   
HIS               Cloud-Related    10,121.35  12,121.67  14,539.92  17,386.33   
                  non-Cloud        41,784.55  43,095.06  43,234.31  44,445.71   
Horizontal BPO    Cloud-Related    11,560.12  14,719.42  19,763.21  23,711.19   
                  non-Cloud       182,996.21 188,811.79 184,365.05 186,850.29   
IT E&T            Cloud-Related     5,690.29   6,560.99   7,728.12   9,023.01   
                  non-Cloud        12,840.17  12,811.42  11,116.75  10,907.35   
ITC               Cloud-Related     6,901.00   8,420.38   9,498.42  11,162.11   
                  non-Cloud        31,952.23  32,682.99  30,663.03  30,364.61   
ITO               Cloud-Related    21,052.64  24,942.34  27,914.67  33,074.93   
                  non-Cloud        77,689.49  74,298.22  67,962.35  62,629.43   
NC&I              Cloud-Related     5,320.62   6,242.89   7,218.05   8,542.02   
                  non-Cloud        40,085.21  41,562.53  42,496.54  41,767.39   
NEOS              Cloud-Related     4,532.93   6,125.72   7,782.27  10,163.58   
                  non-Cloud        48,232.13  49,284.96  47,425.85  46,922.09   
SD&S              Cloud-Related     8,449.37   9,887.90  11,326.11  13,641.35   
                  non-Cloud        66,094.56  67,185.81  65,672.39  64,999.61   
SI                Cloud-Related    22,344.42  26,701.61  29,362.28  34,283.50   
                  non-Cloud       109,675.63 111,390.12 102,759.22 101,679.80   

Year                                    2022       2023       2024       2025  
Foundation Market Cloud/Non-Cloud                                              
AM                Cloud-Related    19,079.44  22,027.83  24,929.32  27,607.34  
                  non-Cloud        58,298.37  57,989.73  57,678.50  57,557.05  
BC                Cloud-Related    57,364.54  65,672.05  73,353.52  83,601.64  
                  non-Cloud        86,966.71  88,594.45  91,603.38  92,687.79  
CAD               Cloud-Related    14,197.30  16,379.96  18,647.57  21,085.15  
                  non-Cloud        40,025.86  40,546.59  41,079.23  41,333.67  
HAM               Cloud-Related     6,921.93   7,832.22   8,779.98   9,705.09  
                  non-Cloud        11,647.24  11,831.96  11,990.14  12,215.18  
HD&S              Cloud-Related    15,759.99  17,282.44  19,019.41  20,805.60  
                  non-Cloud        52,546.95  53,030.69  53,173.54  54,224.43  
HIS               Cloud-Related    20,544.24  23,937.98  27,574.55  31,589.82  
                  non-Cloud        45,540.26  46,578.03  47,621.23  48,663.80  
Horizontal BPO    Cloud-Related    26,677.66  30,594.84  34,110.26  38,056.64  
                  non-Cloud       191,429.43 195,778.27 201,043.76 206,688.79  
IT E&T            Cloud-Related    10,292.53  11,644.48  12,980.58  14,213.49  
                  non-Cloud        10,628.59  10,304.32  10,037.18   9,941.34  
ITC               Cloud-Related    13,098.29  15,247.52  17,419.45  19,548.69  
                  non-Cloud        30,161.89  29,

Year                               2018  2019  2020  2021  2022  2023  2024  \
Foundation Market Cloud/Non-Cloud                                             
AM                Cloud-Related    0.02  0.02  0.01  0.01  0.03  0.06  0.08   
                  non-Cloud        0.02  0.02  0.02  0.02  0.02  0.01 -0.00   
BC                Cloud-Related   -0.47 -0.45 -0.52 -0.52 -0.52 -0.51 -0.49   
                  non-Cloud        0.17  0.18  0.27  0.31  0.36  0.40  0.42   
CAD               Cloud-Related    0.01  0.01  0.01  0.02  0.03  0.04  0.06   
                  non-Cloud        0.01  0.01  0.01  0.02  0.02  0.02  0.03   
HAM               Cloud-Related    0.02  0.02  0.01  0.01  0.02  0.03  0.04   
                  non-Cloud        0.02  0.02  0.02  0.02  0.01  0.00 -0.00   
HD&S              Cloud-Related    0.01  0.01  0.01  0.01  0.01  0.02  0.03   
                  non-Cloud        0.01  0.01  0.01  0.01  0.01  0.01  0.00   
HIS               Cloud-Related    0.04  0.04  0.03  0.03  0.06  0.08  0.09   
                  non-Cloud        0.04  0.04  0.04  0.04  0.04  0.03  0.02   
Horizontal BPO    Cloud-Related    0.02  0.02  0.02  0.02  0.05  0.08  0.11   
                  non-Cloud        0.02  0.02  0.02  0.02  0.01  0.01  0.00   
IT E&T            Cloud-Related   -0.00 -0.00 -0.00 -0.01 -0.01 -0.01 -0.01   
                  non-Cloud        0.00  0.00  0.00  0.02  0.02  0.03  0.04   
ITC               Cloud-Related    0.02  0.01  0.01  0.01  0.01  0.02  0.02   
                  non-Cloud        0.02  0.02  0.02  0.02  0.03  0.03  0.03   
ITO               Cloud-Related    0.01  0.01  0.00 -0.01  0.00  0.02  0.03   
                  non-Cloud        0.01  0.01  0.01  0.01  0.01  0.00 -0.01   
NC&I              Cloud-Related    0.01  0.01  0.01  0.01  0.02  0.03  0.03   
                  non-Cloud        0.01  0.01  0.01  0.01  0.01  0.01  0.01   
NEOS              Cloud-Related    0.01  0.01  0.00  0.00  0.03  0.06  0.07   
                  non-Cloud        0.01  0.01  0.01  0.01  0.00 -0.01 -0.02   
SD&S              Cloud-Related    0.02  0.02  0.02  0.01  0.02  0.03  0.05   
                  non-Cloud        0.01  0.01  0.01  0.00  0.01  0.03  0.04   
SI                Cloud-Related    0.02  0.02  0.02  0.02  0.02  0.03  0.04   
                  non-Cloud        0.01  0.01  0.01  0.02  0.02  0.01  0.01   

Year                               2025  
Foundation Market Cloud/Non-Cloud        
AM                Cloud-Related    0.11  
                  non-Cloud       -0.01  
BC                Cloud-Related   -0.48  
                  non-Cloud        0.46  
CAD               Cloud-Related    0.07  
                  non-Cloud        0.03  
HAM               Cloud-Related    0.06  
                  non-Cloud       -0.01  
HD&S              Cloud-Related    0.05  
                  non-Cloud       -0.00  
HIS               Cloud-Related    0.10  
                  non-Cloud        0.02  
Horizontal BPO    Cloud-Related    0.14  
                  non-Cloud       -0.01  
IT E&T            Cloud-Related   -0.01  
                  non-Cloud        0.05  
ITC               Cloud-Related    0.03  
                  non-Cloud        0.03  
ITO               Cloud-Related    0.04  
                  non-Cloud       -0.02  
NC&I              Cloud-Related    0.05  
                  non-Cloud        0.00  
NEOS              Cloud-Related    0.09  
                  non-Cloud       -0.04  
SD&S              Cloud-Related    0.07  
                  non-Cloud        0.04  
SI                Cloud-Related    0.06  
                  non-Cloud        0.01

In [52]:
# df_pivot = pd.pivot_table(
#     df,
#     values = 'Value', 
#     index = 'Region', 
#     columns = 'Year', 
#     margins = True, 
#     aggfunc = [np.sum],)

# wb = xw.Book()
# sht = wb.sheets('Sheet1')
# sht.range(5,1).value = df_pivot
# df_prev_cycle_pivot = pd.pivot_table(
#     df_prev_cycle, 
#     values = 'Value', 
#     index = 'Region', 
#     columns = 'Year', 
#     margins = False, 
#     aggfunc = [np.sum])

# df_pivot_grwth = df_pivot.pct_change(axis = 1)
# df_prev_cycle_pivot_grwth = df_prev_cycle_pivot.pct_change(axis = 1)

In [ ]:
# pd.set_option('display.float_format', '{:,.0f}'.format)  ### set display format
# display(df_pivot)

# pd.set_option('display.float_format', '{:,.2%}'.format)
# display(df_pivot_grwth)

# pd.set_option('display.float_format', '{:,.0f}'.format)
# display(df_prev_cycle_pivot)

# pd.set_option('display.float_format', '{:,.2%}'.format)
# display(df_prev_cycle_pivot_grwth)

### Read results into dataframes

In [116]:

df = result_ww_source
# df = df.drop(["Value"], inplace = True, axis = 1)
df_pivot = (dd.pivot(df)).groupby('Region').sum()
# df_pivot.groupby('Region').sum()
df_pivot_sum = df_pivot.sum(axis = 0)
df_pivot.append(df_pivot_sum, ignore_index=False)

TypeError: Can only append a Series if ignore_index=True or if the Series has a name

In [24]:
df_pivot = dd.pivot(df)
df_pivot = df_pivot.groupby(['Region']).sum()
df_pivot.loc['Total'] = df_pivot.sum()
df_pivot

wb = xw.Book()
sht = wb.sheets('Sheet1')
sht.range('A5').value = df_pivot

In [47]:
print(df_pivot.shape)
print(df_pivot.shape[0])
print(df_pivot.shape[1])

(10, 8)
10
8


## Customer Care BPO

In [8]:
df = result
df_ccbpo = df[(df["Sub-Foundation Market"] == "Customer Care Business Process Outsourcing") & df.Year.isin(['2019', '2020', '2021', '2022', '', '2023', '2024', '2025'])]
df_ccbpo_pivot = pd.pivot_table(
    df_ccbpo, 
    values = 'Value', 
    index = 'Year', 
    columns = 'Region', 
    margins = True, 
    aggfunc = [np.sum])

pd.set_option('display.float_format', '{:,.0f}'.format)  ### set display format
display(df_ccbpo_pivot)

# percentage change table
pd.set_option('display.float_format', '{:,.1%}'.format)  ### set display format
display(df_ccbpo_pivot.pct_change())

sum                                                       \
Region  APeJC Canada Central & Eastern Europe  Japan Latin America   
Year                                                                 
2019    4,496  1,524                      365  4,068         6,289   
2020    4,665  1,541                      362  4,352         5,556   
2021    5,187  1,702                      409  4,499         5,732   
2022    5,462  1,751                      449  4,678         5,816   
2023    5,752  1,788                      484  4,870         6,076   
2024    6,060  1,824                      519  5,055         6,368   
2025    6,390  1,860                      555  5,227         6,652   
All    38,011 11,991                    3,143 32,751        42,489   

                                                                   
Region Middle East & Africa    PRC     USA Western Europe     All  
Year                                                               
2019                  1,613  3,101  43,309         10,925  75,690  
2020                  1,550  3,160  44,090         10,765  76,040  
2021                  1,666  3,676  45,647         11,540  80,058  
2022                  1,747  3,880  47,306         11,832  82,922  
2023                  1,856  4,307  48,966         12,152  86,251  
2024                  1,981  4,744  50,582         12,475  89,608  
2025                    833  5,279  52,491         12,808  92,096  
All                  11,246 28,147 332,390         82,497 582,664

sum                                                       \
Region  APeJC Canada Central & Eastern Europe  Japan Latin America   
Year                                                                 
2019      NaN    NaN                      NaN    NaN           NaN   
2020     3.7%   1.1%                    -0.8%   7.0%        -11.6%   
2021    11.2%  10.5%                    13.0%   3.4%          3.2%   
2022     5.3%   2.9%                     9.8%   4.0%          1.5%   
2023     5.3%   2.1%                     7.9%   4.1%          4.5%   
2024     5.4%   2.0%                     7.2%   3.8%          4.8%   
2025     5.4%   2.0%                     6.9%   3.4%          4.5%   
All    494.9% 544.5%                   466.2% 526.5%        538.7%   

                                                                 
Region Middle East & Africa    PRC    USA Western Europe    All  
Year                                                             
2019                    NaN    NaN    NaN            NaN    NaN  
2020                  -3.9%   1.9%   1.8%          -1.5%   0.5%  
2021                   7.5%  16.3%   3.5%           7.2%   5.3%  
2022                   4.9%   5.6%   3.6%           2.5%   3.6%  
2023                   6.2%  11.0%   3.5%           2.7%   4.0%  
2024                   6.7%  10.2%   3.3%           2.7%   3.9%  
2025                 -57.9%  11.3%   3.8%           2.7%   2.8%  
All                1,249.7% 433.2% 533.2%         544.1% 532.7%

In [31]:
result

,Region,Country,Primary Market,Secondary Market,Foundation Market,Sub-Foundation Market,Year,Value,Value (Constant)
0,LA,Brazil,Project Oriented,IT Consulting,ITC,IT Consulting,2021,"41,898.30%","42,213.48%"
1,APeJC,Taiwan,Managed Services,Business Process Outsourcing,Horizontal BPO,Procurement Business Process Outsourcing,2021,"1,898.01%","1,899.85%"
2,WE,Portugal,Support Services,IT Education and Training,IT E&T,IT Education and Training,2021,"2,369.68%","2,369.51%"
3,CEE,Czech Republic,Project Oriented,Systems and Network Implementations,NC&I,Network Consulting and Integration,2021,"18,101.36%","18,163.15%"
4,WE,Denmark,Project Oriented,Application Development,CAD,Custom Application Development,2021,"40,017.94%","40,016.33%"
...,...,...,...,...,...,...,...,...,...
11881,WE,Sweden,Managed Services,Business Process Outsourcing,Horizontal BPO,Procurement Business Process Outsourcing,2024,"2,415.64%","2,415.64%"
11882,USA,USA,Project Oriented,Business Consulting,BC,Business Consulting,2024,"9,531,625.31%","9,531,625.31%"
11883,LA,Brazil,Managed Services,Business Process Outsourcing,Horizontal BPO,Customer Care Business Process Outsourcing,2024,"295,568.12%","295,568.12%"
11884,CEE,Poland,Managed Services,Business Process Outsourcing,Horizontal BPO,Procurement Business Process Outsourcing,2024,"10,913.38%","10,913.38%"
